In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class DataCleaning(object):
    def process_item(self, item, spider):
        ngay = self.string_standardize(dict(item)["ngay"])
        loai = self.string_standardize(dict(item)["loai"])
        giamua = self.string_standardize(dict(item)["gia_mua"])
        giaban = self.string_standardize(dict(item)["gia_ban"])
        giamua = giamua.replace(',','')
        giaban = giaban.replace(',','')
        return {'ngay': ngay, 'loai_vang': loai, 'gia_mua_vao': giamua, 'gia_ban_ra': giaban}
   
    def string_standardize(self, text):
        text = text.strip("\n")
        text = text.strip()
        return text

In [3]:
# Định nghĩa lớp kế thừa từ Spider
class GiaVangSpider(scrapy.Spider):
    ngay = '2023-09-15'

    name = "vang24h_spider"
    start_urls = [f'https://www.24h.com.vn/gia-vang-hom-nay-c425.html?d={ngay}']

    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.DataCleaning': 1
        },
        'FEED_FORMAT': 'csv',
        'FEED_URI': f'gia-vang-{ngay}.csv',
        'LOG_ENABLED': False,
    }

    def parse(self, response):
        ngay = response.css('span.inputDate::text').get()
        bang_gia = response.css('table.gia-vang-search-data-table')
        for tr in bang_gia.css('tbody > tr'):
            loai = tr.css('h2::text').get(default='')
            ds_gia = tr.css('span.fixW::text').getall()
            gia_mua = ds_gia[0]
            gia_ban = ds_gia[1]
            yield {'ngay': ngay, 'loai': loai, 'gia_mua': gia_mua, 'gia_ban': gia_ban}

In [4]:
process = CrawlerProcess()
process.crawl(GiaVangSpider)
process.start()

2023-09-15 20:53:26 [scrapy.utils.log] INFO: Scrapy 2.10.1 started (bot: scrapybot)
2023-09-15 20:53:26 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.1.2 1 Aug 2023), cryptography 41.0.3, Platform Windows-10-10.0.19045-SP0
2023-09-15 20:53:26 [scrapy.addons] INFO: Enabled addons:
[]
2023-09-15 20:53:26 [scrapy.crawler] INFO: Overridden settings:
{'LOG_ENABLED': False}
